In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import json
import wave
# sudo apt install portaudio19-dev python3-pyaudio
import pyaudio
import torchaudio 
from pprint import pprint
from scipy.io import wavfile
import io
import wave
# import soundfile as sf

# Data Prepare
## 1.1 torchaudio data

data: 
- name: LibriSpeech [Panayotov et al., 2015] dataset.
- info:
- slice[idx]: clean-data:
    - Waveform(Tensor)
    - Sample rate(int): sampled at 16 kHz. 
    - Transcript(str):  vedio -> text
    - Speaker ID(int)
    - Chapter ID(int)
    - Utterance ID(int)
- get_metadata(idx):
    - Path to audio(str: fac-file), Sample rate(int), Transcript(str), Speaker ID(int), Chapter ID(int), Utterance ID(int)

In [2]:
librispeech_path = '/home/scc/Downloads/Datas/speech2textData'
yesno_data = torchaudio.datasets.LIBRISPEECH(librispeech_path, download=False)
wave, sample_rate, trans, spk_id, chap_id, u_id = yesno_data[4]
wave.shape

torch.Size([1, 200240])

In [3]:
yesno_data.get_metadata(0)

('train-clean-100/103/1240/103-1240-0000.flac',
 16000,
 'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK',
 103,
 1240,
 0)

In [10]:
import torch

torch.cuda.is_available()

False

## 1.2 collect real data

In [4]:
data_root_dir = Path('/home/scc/Downloads/Datas/speech2textData')
data_dir = data_root_dir.joinpath('genshin-voice-v3.5-mandarin/data')
label_path = data_root_dir.joinpath('result.json')

In [5]:
with label_path.open(mode='r') as f:
    label_res = json.load(f)

In [6]:
list(label_res.keys())[:10]

['00000ad748e4defa',
 '00003b433f092c26',
 '00003e876928ec1f',
 '0000519d3a5004e8',
 '0000b124af136a62',
 '0000cb0ced63faaa',
 '0000d5d3e270abc5',
 '0000d6fff0f3a1de',
 '0000f9f7ada1c623',
 '00010b7659af6235']

In [7]:
file = [i for i in data_dir.iterdir()][0]
print(file)
df = pd.read_parquet(file)
df.columns

/home/scc/Downloads/Datas/speech2textData/genshin-voice-v3.5-mandarin/data/train-00038-of-00067-dc30261178723032.parquet


Index(['audio', 'language', 'npcName', 'text', 'type'], dtype='object')

In [8]:
def bytes2pt(b_in, use_tmp_file=False, tmp_file='./tmp.wav'):
    # method-1
    b_arr = bytearray(b_in)
    np_arr = np.array(b_arr, dtype=np.int16)
    if use_tmp_file:
        wavfile.write(tmp_file, 16000, np_arr)
        waveform, sample_rate = torchaudio.load(tmp_file)
        return waveform, sample_rate 

    # method-2
    bytes_wav = bytes()
    byte_io = io.BytesIO(bytes_wav)
    wavfile.write(byte_io, 16000, np_arr)
    # output_wav = byte_io.read()
    # waveform, sample_rate = torchaudio.load(io.BytesIO(output_wav))
    waveform, sample_rate = torchaudio.load(byte_io)
    return waveform, sample_rate 


display(df.iloc[0])
for k, v in df.iloc[0]['audio'].items():
    print(k, '=>', v)
bytes_in = df.iloc[0]['audio']['bytes']
waveform1, sample_rate = bytes2pt(bytes_in, True)
waveform2, sample_rate = bytes2pt(bytes_in, False)
print(waveform1.shape, waveform2.shape, sample_rate, (waveform1 != waveform2).sum())
b_arr = bytearray(bytes_in)
np_arr = np.array(b_arr, dtype=np.int16)

audio       {'bytes': b'RIFF\xa4\t\x01\x00WAVEfmt \x10\x00...
language                                                  CHS
npcName                                                   珐露珊
text                                                     敢惹我？
type                                                   Fetter
Name: 0, dtype: object

bytes => b'RIFF\xa4\t\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00data\x80\t\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\x02\x00\xfe\xff\x01\x00\xff\xff\x00\x00\x02\x00\xff\xff\x01\x00\x00\x00\xfe\xff\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\x02\x00\xfe\xff\x01\x00\x01\x00\x02\x00\x01\x00\x01\x00\x01\x00\x01\x00\x01\x00\x01\x00\x01\x00\x00\x00\x00\x00\xff\xff\x03\x00\x00\x00\x02\x00\x01\x00\x01\x00\x01\x00\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\xff\xff\x01\x00\xfe\xff\x05\x00\xfe\xff\x05\x00\xfe\xff\x01\x00\x02\x00\x00\x00\x03\x00\xff\xff\x00\x00\xfe\xff\x03\x00\xfe\xff\x03\x00\x02\x00\x03\x00\xff\xff\x00\x00\xfe\xff\x03\x00\xfe\xff\x03\x00\x02\x00\xff\xff\x01\x00\x00\x00\x02\x00\xfd\xff\x02\x00\x01\x00\x02\x00\xfe\xff\x01\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xfe\xff\x02\x00\xff

/home/scc/anaconda3/envs/speech2text/lib/python3.10/site-packages/torchaudio/backend/sox_io_backend.py:235: UserWarning: File-like object support in sox_io backend is deprecated, and will be removed in v2.1. See https://github.com/pytorch/audio/issues/2950 for the detail.Please migrate to the new dispatcher, or use soundfile backend.
  warnings.warn(_deprecation_message)


In [12]:
from chardet import detect
bt_a = bytes_in
print(detect(bt_a[:40]))
print(bt_a[:40],  'RIFF', int('a4', base=16), int('01', base=16), int('00', base=16), "WAVEfmt")
# bt_a[:16].decode()
int('b', base=16)
# RIFF164 10WAVEfmt  16000 1010 128  11b 187000w 1020160

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
b'RIFF\xa4\t\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00data' RIFF 164 1 0 WAVEfmt


11

In [20]:
import re

re_c = re.compile(r'[a-zA-Z]')

for i in range(10):
    bt_a = df.iloc[i]['audio']['bytes']
    b_arr = bytearray(bt_a)
    np_arr = np.array(b_arr, dtype=np.int16)
    info_list = []
    for bi in bytes_in[:40]:
        if re_c.match(chr(bi)):
            info_list.append(chr(bi))
            continue
        info_list.append(bi)
        # info_list.append(chr(bi))
        
    print(f'i={i}',''.join(str(i) for i in info_list), np_arr[:40], b_arr[:40])

i=0 RIFF164910WAVEfmt32160001010128187000w1020160data [ 82  73  70  70 164   9   1   0  87  65  86  69 102 109 116  32  16   0
   0   0   1   0   1   0 128 187   0   0   0 119   1   0   2   0  16   0
 100  97 116  97] bytearray(b'RIFF\xa4\t\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00data')
i=1 RIFF164910WAVEfmt32160001010128187000w1020160data [ 82  73  70  70 164 232   0   0  87  65  86  69 102 109 116  32  16   0
   0   0   1   0   1   0 128 187   0   0   0 119   1   0   2   0  16   0
 100  97 116  97] bytearray(b'RIFF\xa4\xe8\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x00\x02\x00\x10\x00data')
i=2 RIFF164910WAVEfmt32160001010128187000w1020160data [ 82  73  70  70 164 244   1   0  87  65  86  69 102 109 116  32  16   0
   0   0   1   0   1   0 128 187   0   0   0 119   1   0   2   0  16   0
 100  97 116  97] bytearray(b'RIFF\xa4\xf4\x01\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80\xbb\x00\x00\x00w\x01\x0

In [ ]:
def bytes2pt(b_in, use_tmp_file=False, tmp_file='./tmp.wav'):
    # method-1
    if use_tmp_file:
        b_arr = bytearray(b_in)
        np_arr = np.array(b_arr, dtype=np.int16)
        wavfile.write(tmp_file, 16000, np_arr)
        waveform, sample_rate = torchaudio.load(tmp_file)
        return waveform, sample_rate 
    # method-2
    bytes_wav = bytes()
    byte_io = io.BytesIO(bytes_wav)
    wavfile.write(byte_io, 16000, np.frombuffer(samples_dict['bytes'], dtype=np.int16))
    output_wav = byte_io.read()
    waveform, sample_rate = torchaudio.load(io.BytesIO(output_wav))
    return waveform, sample_rate 